In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tqdm
import ray
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from scipy import sparse
import logging
from functools import lru_cache
from mol2vec.features import mol2alt_sentence, sentences2vec
from gensim.models.word2vec import Word2Vec

In [2]:
ray.init()

2021-05-27 10:46:52,595	INFO services.py:1267 -- View the Ray dashboard at http://127.0.0.1:8266


{'node_ip_address': '172.31.35.146',
 'raylet_ip_address': '172.31.35.146',
 'redis_address': '172.31.35.146:11768',
 'object_store_address': '/tmp/ray/session_2021-05-27_10-46-51_604266_7393/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-05-27_10-46-51_604266_7393/sockets/raylet',
 'webui_url': '127.0.0.1:8266',
 'session_dir': '/tmp/ray/session_2021-05-27_10-46-51_604266_7393',
 'metrics_export_port': 51008,
 'node_id': '01002931e3670a31762bbc60116c7fbc5a1638d85e0f76c9ddaf1dbd'}

In [5]:
NUM_CHUNKS = 10

In [6]:
USE_EMBEDDINGS = False

In [7]:
!ls /mnt/efs/AmpC_data/

AmpC_screen_table.csv  AmpC_screen_table.csv.zip


In [8]:
RECEPTOR = "AmpC"
DATA_DIR = "/mnt/efs/AmpC_data"
INPUT_DATA = f"{DATA_DIR}/{RECEPTOR}_screen_table.csv"

MODEL_PATH = "/home/ubuntu/mol2vec/examples/models/model_300dim.pkl"
UNCOMMON = "UNK"

In [9]:
@lru_cache(maxsize=2)
def get_data():
    ligands_df = pd.read_csv(INPUT_DATA)
    ligands_df = ligands_df[ligands_df["dockscore"] != "no_score"]
    
    return ligands_df

In [13]:
@lru_cache(maxsize=2)
def get_w2v_model():
    word2vec_model = Word2Vec.load(MODEL_PATH)
    word2vec_model.wv.init_sims()
    return word2vec_model

In [18]:
@ray.remote
def create_fingerprint(smiles, score, i):
    if i % 10000 == 0:
        logging.basicConfig(level=logging.INFO)
        logging.info(i)
        
    mol = Chem.MolFromSmiles(smiles)
    pars = { 
        "radius": 2,
        "nBits": 8192,
        "invariants": [],
        "fromAtoms": [],
        "useChirality": False,
        "useBondTypes": True,
        "useFeatures": True,
    }
    fp = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, **pars)
    onbits = list(fp.GetOnBits())

    return onbits, float(score)

In [15]:
@ray.remote
def create_mol_sentence(smiles, score, r, i):
    if i % 10000 == 0:
        logging.basicConfig(level=logging.INFO)
        logging.info(i)
        
    mol = Chem.MolFromSmiles(smiles)
    # smiles = Chem.MolToSmiles(mol)
    
    if not mol:
        return
    
    alt_sentence = mol2alt_sentence(mol, radius=r)
    
    return alt_sentence, score

In [16]:
def get_fingerprints(ligands_df, fp_size=8192, smiles_col="smiles", score_col="score"):
    future_values = [create_fingerprint.remote(smiles=smiles, score=score, i=i) for (i, (smiles, score)) in enumerate(zip(ligands_df[smiles_col], ligands_df[score_col]))]
    
    values = [v for v in ray.get(future_values) if v]
    all_bits, scores = zip(*values)
    
    row_idx = []
    col_idx = []
    for i, bits in enumerate(all_bits):
        # these bits all have the same row:
        row_idx += [i] * len(bits)
        #and the column indices of those bits:
        col_idx += bits
    
    # generate a sparse matrix out of the row,col indices:
    unfolded_size = 8192
    fingerprint_matrix = sparse.coo_matrix((np.ones(len(row_idx)).astype(bool), 
                                           (row_idx, col_idx)), 
                                           shape=(max(row_idx)+1, unfolded_size))
    
    # convert to csr matrix, it is better:
    fingerprint_matrix =  sparse.csr_matrix(fingerprint_matrix)

    return fingerprint_matrix, scores

In [19]:
def get_embeddings(ligands_df, model, radius=1):
    future_values = [create_mol_sentence.remote(smiles=smiles, score=score, r=radius, i=i) for (i, (smiles, score)) in enumerate(zip(ligands_df["smiles"], ligands_df["score"]))]
    
    values = [v for v in ray.get(future_values) if v]
    mol_sentences, scores = zip(*values)

#     vectors = sentences2vec(sentences=mol_sentences, model=model, unseen=UNCOMMON)

    return mol_sentences, scores

In [10]:
ligands_df = get_data()

/home/ubuntu/anaconda3/envs/dockop/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [17]:
ligands_df = ligands_df[ligands_df["dockscore"] != "no_score"]

In [ ]:
# word2vec_model = get_w2v_model()

In [ ]:
start = time.time()
for i, df_chunk in enumerate(np.array_split(ligands_df, NUM_CHUNKS)):
    if USE_EMBEDDINGS:
        print("Generating mol2vec embeddings...")
        embeddings, scores = get_embeddings(ligands_df=df_chunk, model=word2vec_model, radius=1)
        vectors = sentences2vec(sentences=embeddings, model=word2vec_model, unseen=UNCOMMON)
        np.save(f"{DATA_DIR}/{RECEPTOR}_embeddings_{i}.npy", vectors)
        np.save(f"{DATA_DIR}/{RECEPTOR}_embedding_scores_{i}.npy", np.array(scores))
    else:
        print("Generating Morgan Fingerprints...")
        fingerprint_matrix, scores = get_fingerprints(ligands_df=df_chunk, score_col="dockscore")
        sparse.save_npz(f"{DATA_DIR}/{RECEPTOR}_fingerprints_{i}.npz", fingerprint_matrix)
        np.save(f"{DATA_DIR}/{RECEPTOR}_scores_{i}.npy", np.array(scores))
    print(f"Chunk {i} took: {(time.time() - start)/60} mins")
    
print(f"Dataset took: {(time.time() - start)/60} mins")

Generating Morgan Fingerprints...


(pid=10675) INFO:root:0
(pid=10665) INFO:root:10000
(pid=10654) INFO:root:20000
(pid=10672) INFO:root:30000
(pid=10673) INFO:root:40000
(pid=10664) INFO:root:50000
(pid=10670) INFO:root:60000
(pid=10669) INFO:root:70000
(pid=10655) INFO:root:80000
(pid=10690) INFO:root:90000
(pid=10675) INFO:root:100000
(pid=10703) INFO:root:110000
(pid=10690) INFO:root:120000
(pid=10659) INFO:root:130000
(pid=10699) INFO:root:140000
(pid=10673) INFO:root:150000
(pid=10670) INFO:root:160000
(pid=10711) INFO:root:170000
(pid=10667) INFO:root:180000
(pid=10672) INFO:root:190000
(pid=10709) INFO:root:200000
(pid=10661) INFO:root:210000
(pid=10700) INFO:root:220000
(pid=10663) INFO:root:230000
(pid=10672) INFO:root:240000
(pid=10675) INFO:root:250000
(pid=10666) INFO:root:260000
(pid=10655) INFO:root:270000
(pid=10658) INFO:root:280000
(pid=10690) INFO:root:290000
(pid=10665) INFO:root:300000
(pid=10675) INFO:root:310000
(pid=10675) INFO:root:320000
(pid=10704) INFO:root:330000
(pid=10709) INFO:root:340000